In [1]:
import pandas as pd

# 读取 train_essays.csv 和 train_prompts.csv 文件
essays_df = pd.read_csv('llm-detect-ai-generated-text/train_essays.csv')
prompts_df = pd.read_csv('llm-detect-ai-generated-text/train_prompts.csv')

# 使用 prompt_id 字段将两个数据集进行连接
merged_df = essays_df.merge(prompts_df, on='prompt_id', how='left')
merged_df['text'] = merged_df['prompt_name'] + '\n\n' + merged_df['instructions'] + '\n\n' + merged_df['text']
merged_df.drop(merged_df.columns[[4, 5, 6]],axis=1, inplace=True)

merged_df.to_csv('llm-detect-ai-generated-text/train_merged.csv', index=False)

In [2]:
txtdata = []
classes = []
for row in merged_df.itertuples():
    txtdata.append(row[3])
    classes.append(row[4])

In [3]:
from sklearn import *

# Split the data set using the train_test_split method
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(txtdata, classes, test_size=0.2, random_state=42)

In [4]:
# Load Vector Model
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")

In [5]:
# Get sentence vector
train_feature = model.encode(train_data, show_progress_bar=True)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [6]:
# learn logistic regression classifier using CV
from numpy import *
logreg = linear_model.LogisticRegressionCV(Cs=logspace(-4,4,20), cv=5, n_jobs=-1)
logreg.fit(train_feature, train_labels)

print("w=", logreg.coef_)
print("b=", logreg.intercept_)

/Users/Shared/anaconda/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlo

w= [[-2.60807191e-05 -1.54172540e-05 -2.67677718e-05  3.45501979e-05
  -6.05386623e-06 -2.14651226e-06 -2.76371231e-05 -7.89148591e-06
   3.94534724e-05 -2.60347916e-05  6.21360014e-06 -7.24760733e-05
   6.88243442e-06 -2.92709671e-05 -3.87525379e-05  2.21491001e-05
   4.49172869e-05 -6.33900735e-06  5.11629084e-06  1.81685409e-05
  -1.30361654e-06 -1.81473724e-05  5.86532996e-05  7.45909083e-05
   3.42768295e-05 -4.38578423e-06  6.34814460e-05 -4.15062150e-06
  -1.27130738e-05 -2.49552200e-06  8.75795178e-07  7.30583485e-05
   1.04945863e-05  8.60243768e-06  9.89390312e-06 -1.10543935e-04
  -2.59772575e-05  2.18467751e-05 -1.14404878e-05 -4.08437417e-05
   4.23412529e-06 -5.13370081e-05  1.13254422e-05 -4.56283071e-05
  -1.20143992e-05 -2.64178920e-05 -4.08226337e-05  4.02928405e-05
  -2.01886893e-05 -2.03081648e-05 -3.22559651e-05 -3.64169829e-05
   2.58273121e-05 -3.54704972e-05  1.23583446e-04  1.08028704e-05
  -4.63866417e-05  3.74205383e-06  1.32377568e-05 -4.16959728e-05
  -3.28

In [7]:
test_feature = model.encode(test_data, show_progress_bar=True)

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
# predict from the model
predY = logreg.predict(test_feature)

# calculate accuracy
acc = metrics.accuracy_score(test_labels, predY)
print("test accuracy=", acc)

test accuracy= 0.9963768115942029
